This Notebook is classifiy the input image to get built-up land maps.<br>

Specifically, we:<br>
1) loop through each year-range (1990-2019 at 3-year intervals);<br>
2) loop through each seed number (0-9) to create 10 classifications with diff samples;
3) export classification to Assest

In [1]:
import ee
import datetime
import os
import itertools
import sys
import re

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import geemap

import subprocess
from subprocess import PIPE
import ast

In [2]:
ee.Initialize()

In [3]:
# append upper folder into sys-path during run time so we can
# import our own moduls
sys.path.append('..')

In [4]:
from BackGround_modules.Class_2_Classify_Fourier_Img   import Classification
from BackGround_modules.Class_3_Calculate_the_accuracy import Accuracy_assesment

##### Prepare basic parameters

In [5]:
# define the years to be classified
year_name  = [f'{i}_{i+2}' for i in range(1990,2020,3)]

# define the number of total periods
num_invarient = 10

# import north_china_plain boundary
North_China_Plain = ee.FeatureCollection("users/wangjinzhulala/North_China_Plain_Python/Boundary_shp/North_China_Plain_Boundary")

In [6]:
year_name

['1990_1992',
 '1993_1995',
 '1996_1998',
 '1999_2001',
 '2002_2004',
 '2005_2007',
 '2008_2010',
 '2011_2013',
 '2014_2016',
 '2017_2019']

##### Prepare input_image for classification

Some additional treatment for the last two periods of Landsat and Sentinel imgs

In [7]:
# import Landsat img
Landsat_img = [ee.Image(f"users/wang8052664/Cloud_Free_Img/Landsat_cloud_free_{year}")\
                 .clip(North_China_Plain)  for year in year_name]

In [8]:
# ________________________________________import Sentinel cloud free img______________

# we do not store the Sentinel img to gee Asset because it requires more space than allowed,
# so we use Sentinel on-the-fly here

# first introduce a cloud masking function
def maskS2clouds(image):
    qa = image.select('QA60')

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask  = 1 << 10;
    cirrusBitMask = 1 << 11;

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    # Return the masked and scaled data, without the QA bands.
    return image.updateMask(mask)\
                 .select("B.*")\
                 .copyProperties(image, ["system:time_start"])



# here composite Sentinel-2 multispectrum image of year [2015-2017] for classification of [2014-2016]
#            and Sentinel-2 multispectrum image of year [2018-2019] for classification of [2017-2019]
# because the composition of S2 image at year [2014-2016] are not fully cloud-free
Sentinel_year_range = [("2015-01-01","2017-12-31"),("2018-01-01","2019-12-31")]

Sentinel_img =  [ee.ImageCollection("COPERNICUS/S2")\
                      .filterBounds(North_China_Plain)\
                      .filterDate(*S_t)\
                      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5))\
                      .map(lambda x: maskS2clouds(x))\
                      .median() for S_t in Sentinel_year_range] 

In [9]:
# combine Landsat/Sentinel imgs together, note for the year 2014_2016, 2017-2019,
# we added the sensor name before each landsat/Sentine bands, because otherwise
# it not possible to distinguash bands (since bands of Landsat and Sentinel are
# all named as B* )

Landsat_2014_2019  = [img.rename([f'Landsat_{band}' for band in  img.bandNames().getInfo()]) for img in Landsat_img[-2:]]
Sentinel_2014_2019 = [img.rename([f'Sentinel_{band}' for band in  img.bandNames().getInfo()]) for img in Sentinel_img]

Landsat_Sentinel  = [ee.Image(img) for img in zip(Landsat_2014_2019,Sentinel_2014_2019)]

In [10]:
# put original Landsat and Landsat_Sentinel_2014_2019 to one list
Landsat_Sentinel_input = Landsat_img[:-2] + Landsat_Sentinel

Now prepare other input imgs and stack all img together

In [11]:
# import Fourier img   
Fourier_img = [ee.Image(f"users/Jinzhu_Deakin/North_China_Plain/Fourier_img_harmonic_3/Fourier_img_{year}_harmonic_3")\
                 .clip(North_China_Plain)  for year in year_name[:5]] + \
              [ee.Image(f"users/wang8052664/North_China_Plain/Fourier_imgs/Fourier_img_{year}_harmonic_3")\
                 .clip(North_China_Plain)  for year in year_name[5:]] 

# Import the Index-mean Img
NDVI_img = [ee.Image(f"users/wensomone666/Jinzhu/Mean_NDVI/Year_{year}_Mean_NDVI").rename('Mean_NDVI')  
            for year in year_name ]
NDBI_img = [ee.Image(f"users/wensomone666/Jinzhu/Mean_NDVI/Year_{year}_Mean_NDBI").rename('Mean_NDBI')  
            for year in year_name ]
EVI_img  = [ee.Image(f"users/wensomone666/Jinzhu/Mean_NDVI/Year_{year}_Mean_EVI").rename('Mean_EVI')  
            for year in year_name ]

# prepare the climate data
Climate_mean = [ee.Image(f"users/wangjinzhulala/North_China_Plain_Python/Climate_data/Mean_{year}")
               .clip(North_China_Plain)  for year in year_name]

# Import DEM/SLOPE Img
DEM   = [ee.Image("USGS/SRTMGL1_003").rename('DEM')] * num_invarient
SLOPE = [ee.Terrain.slope(DEM).rename('SLOPE')] * num_invarient

In [12]:
# stack all Imput_Img together
Stack_img = [ee.Image(img) for img in zip(Landsat_Sentinel_input,
                                          Fourier_img,
                                          NDVI_img,
                                          NDBI_img,
                                          EVI_img,
                                          Climate_mean,
                                          DEM,
                                          SLOPE)]

################ Some extra export for fig making, not useful for this classification ################

In [18]:
# # get the observation rectagles
# # clip = ee.FeatureCollection("users/wangjinzhulala/North_China_Plain_Python/Boundary_shp/Observation_Rectangle_Full")

# # loop through each year to get target imgs
# for year,img in zip(year_name,Stack_img):
    
#     # get imgs
#     Fourier = img.select(['NDBI_cos_1','EVI_sin_2','NDVI_sin_2']).clip(clip)
#     Normalized = img.select(['Mean_NDVI','Mean_NDBI','Mean_EVI']).clip(clip)
    
#     # Landsat 8 has different flase color composite compare to 5/7
#     if int(year[-4:]) <= 2013 :
#         False_color = img.select(['B4','B3','B2']).clip(clip)
#     else:
#         False_color = img.select(['Landsat_B5','Landsat_B4','Landsat_B3']).clip(clip)
        
        
#     # stack imgs together for export    
#     imgs = [Fourier,Normalized,False_color] 
#     img_names = ['Fourier','Normalized','False_color'] 
    
#     # export to CloudStorage
#     for img,name in zip(imgs,img_names):
        
#         # construcnt img name for export
#         img_name = f'{name}_{year}'
        
#         # export
#         task = ee.batch.Export.image.toCloudStorage(image = img,
#                                                     description=img_name,
#                                                     bucket='north_china_plain',
#                                                     fileNamePrefix=img_name,
#                                                     region=clip,
#                                                     scale=30,
#                                                     maxPixels=int(1e13),
#                                                     skipEmptyTiles=True) 
    
#         task.start()
        
#         # print out the process.
#         print(f'Exporting {img_name} successful!')
    

Exporting Fourier_2008_2010 successful!
Exporting Normalized_2008_2010 successful!
Exporting False_color_2008_2010 successful!


##### Prepare input sample points

In [13]:
# Define the path to Sample_pt
path = 'users/wangjinzhulala/North_China_Plain_Python/Sample_extract_img'

# Get the training sample

# Here we use a random colum to select 3/4 of the sample points for classification/assessment
# and leave 1/4 that keep untouched for later accuracy comparision

sample_landsat = [ee.FeatureCollection(f"{path}/Control_sample_ext_img_{year}")\
                     .randomColumn('split', 101)\
                     .filterMetadata('split','greater_than',0.25)
                   for year in year_name[:-2]] 

sample_sentinel = [ee.FeatureCollection(f"{path}/Sentinel_Landsat_reinspect_sample_ext_img_{year}")\
                     .randomColumn('split', 101)\
                     .filterMetadata('split','greater_than',0.25)
                   for year in year_name[-2:]] 
    
    
Training_sample = sample_landsat + sample_sentinel

##### Stack input_img and input_sample together

In [14]:
# first put all inputs together
stack_name   = ['Control'] * num_invarient

stack_year   = year_name
stack_img    = Stack_img 
stack_sample = Training_sample 

# stack all ingredients together
Stack_img_sample = list(zip(stack_name,stack_year,stack_img,stack_sample))

#### Create the classification instances

In [15]:
# import the inbands-combination 
in_band_df = pd.read_csv('../Sub_Process_6_Before_classification_Feature_selection/Result/In_bands_combination.csv')

# selections for in-band combination
in_band_selection =['Spectrum',
                    'Spectrum_Normalize',
                    'Spectrum_Normalize_Fourier',
                    'Spectrum_Normalize_Fourier_Terrain',
                    'Spectrum_Normalize_Fourier_Terrain_Meterology']

In [16]:
# A three-layer iteration to loopthrough [inbands/year/10-randome-split] individualy

Classificatioin_result = []

# The first-layer loop to go througth each inbands
for input_bands in in_band_selection:

    # The second-layer loop to go througth each year-periods
    for i,input_variable in enumerate(Stack_img_sample):

        # Fetch basic parameters
        classificaiton_tpye = input_bands
        year                = input_variable[1]
        input_img           = input_variable[2]
        sample_pt           = input_variable[3]

        # here determine which bands are involved in the classification
        # because the list is a string represented format, here use ast 
        # module to conver it back to a Python list
        in_features         = ast.literal_eval(in_band_df.at[i,input_bands])
        

        # The third-layer to go through each-randomnes
        for seed in range(10):

            #_______________________________Deploy the random-forest classification______________________
            # Instatiate the class with a name.
            classification = Classification(year_name      = year,
                                            Verified_point = sample_pt,
                                            Input_img      = input_img,
                                            Input_band     = in_features,
                                            Tree_num       = 50,
                                            seed           = seed,
                                            classProperty  = 'Built')


            # Get the classified img
            classified_img = classification.classification_img


            # Get the classified samples
            train_sample_classified = classification.Train_sample_classification
            test_sample_classified  = classification.Test_sample_classification

            #__________________________________Add all classification result into a list____________________________
            Classificatioin_result.append((classificaiton_tpye,year,seed,classified_img,test_sample_classified))

In [17]:
# convert classificaiton instances into a dataframe
Classification_instances_df =  pd.DataFrame(Classificatioin_result,
                                            columns=['classificaiton_tpye','year','seed','classified_img',
                                                     'test_sample_classified'])
Classification_instances_df

,classificaiton_tpye,year,seed,classified_img,test_sample_classified
0,Spectrum,1990_1992,0,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
1,Spectrum,1990_1992,1,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
2,Spectrum,1990_1992,2,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
3,Spectrum,1990_1992,3,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
4,Spectrum,1990_1992,4,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
...,...,...,...,...,...
495,Spectrum_Normalize_Fourier_Terrain_Meterology,2017_2019,5,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
496,Spectrum_Normalize_Fourier_Terrain_Meterology,2017_2019,6,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
497,Spectrum_Normalize_Fourier_Terrain_Meterology,2017_2019,7,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."
498,Spectrum_Normalize_Fourier_Terrain_Meterology,2017_2019,8,"ee.Image({\n ""type"": ""Invocation"",\n ""argume...","ee.FeatureCollection({\n ""type"": ""Invocation""..."


#### Add one of the classified img to map

In [18]:
# define parameters of classification for show in map
bands_in_type = 'Spectrum_Normalize_Fourier_Terrain_Meterology'
year_in       = '2017_2019'


# get the classification img
one = Classification_instances_df[(Classification_instances_df['classificaiton_tpye'] == bands_in_type) &
                                  (Classification_instances_df['year'] == year_in) &
                                  (Classification_instances_df['seed'] == 5)]['classified_img'].values[0]

In [19]:
# add the selected classification in map
Map = geemap.Map()
Map.add_basemap('HYBRID')
Map.setCenter(112.8844, 34.6033,10)
Map.addLayer(one,{'min':0,'max':1},'One')
Map

Map(center=[34.6033, 112.8844], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButt…

##### Export the classified_img

In [23]:
for idx,item in Classification_instances_df.iterrows():
     
    Classification_type = item[0]
    year = item[1]
    seed = item[2]
    img  = item[3]
    
    
    export_name = f'{Classification_type}_{year}_{seed}'
    
    #export to GEE asset
    asset_path = f'users/wangjinzhulala/North_China_Plain_Python/classification_img/{Classification_type}_{year}'
    
    task = ee.batch.Export.image.toAsset(   image          = img,
                                            description    = export_name,
                                            assetId        = f'{asset_path}/{export_name}',
                                            region         = North_China_Plain.geometry().bounds(),
                                            scale          = 30,
                                            maxPixels      = int(1e13)) 
    
    task.start()
    
    print(f'Exporting {export_name} successful!')

Exporting Spectrum_2014_2016_0 successful!
Exporting Spectrum_2014_2016_1 successful!
Exporting Spectrum_2014_2016_2 successful!
Exporting Spectrum_2014_2016_3 successful!
Exporting Spectrum_2014_2016_4 successful!
Exporting Spectrum_2014_2016_5 successful!
Exporting Spectrum_2014_2016_6 successful!
Exporting Spectrum_2014_2016_7 successful!
Exporting Spectrum_2014_2016_8 successful!
Exporting Spectrum_2014_2016_9 successful!
Exporting Spectrum_2017_2019_0 successful!
Exporting Spectrum_2017_2019_1 successful!
Exporting Spectrum_2017_2019_2 successful!
Exporting Spectrum_2017_2019_3 successful!
Exporting Spectrum_2017_2019_4 successful!
Exporting Spectrum_2017_2019_5 successful!
Exporting Spectrum_2017_2019_6 successful!
Exporting Spectrum_2017_2019_7 successful!
Exporting Spectrum_2017_2019_8 successful!
Exporting Spectrum_2017_2019_9 successful!
Exporting Spectrum_Normalize_2014_2016_0 successful!
Exporting Spectrum_Normalize_2014_2016_1 successful!
Exporting Spectrum_Normalize_2014_